In [5]:
from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

session = SnorkelSession()
dev_cands = session.query(Spouse).filter(Spouse.split == 1).all()
test_cands = session.query(Spouse).filter(Spouse.split == 2).all()

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]
gold_labels_test = [L[0,0] for L in L_gold_test]

from snorkel.learning.utils import MentionScorer

Instructions for updating:
Use the retry module or similar alternatives.


In [47]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    
# print(len(gold_labels_dev),len(gold_labels_test))
# print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
# print(len(gold_labels_dev))

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(gold_labels_test.count(1),gold_labels_test.count(0))
print(len(gold_labels_dev),len(gold_labels_test))
print()

189 2625
218 2484
2814 2702



In [7]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../../../snorkel/tutorials/glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [58]:
# ####### Discrete ##########

# spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
# family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
#               'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
# family = family | {f + '-in-law' for f in family}
# other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# # Helper function to get last name
# def last_name(s):
#     name_parts = s.split(' ')
#     return name_parts[-1] if len(name_parts) > 1 else None    

# def LF_husband_wife(c):
#     return (1,1) if len(spouses.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_husband_wife_left_window(c):
#     if len(spouses.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (1,1)
#     elif len(spouses.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (1,1)
#     else:
#         return (0,0)
    
# def LF_same_last_name(c):
#     p1_last_name = last_name(c.person1.get_span())
#     p2_last_name = last_name(c.person2.get_span())
#     if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
#         if c.person1.get_span() != c.person2.get_span():
#             return (1,1)
#     return (0,0)

# def LF_no_spouse_in_sentence(c):
#     return (-1,1) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

# def LF_and_married(c):
#     return (1,1) if 'and' in get_between_tokens(c) and 'married' in get_right_tokens(c) else (0,0)
    
# def LF_familial_relationship(c):
#     return (-1,1) if len(family.intersection(get_between_tokens(c))) > 0 else (0,0)

# def LF_family_left_window(c):
#     if len(family.intersection(get_left_tokens(c[0], window=2))) > 0:
#         return (-1,1)
#     elif len(family.intersection(get_left_tokens(c[1], window=2))) > 0:
#         return (-1,1)
#     else:
#         return (0,0)

# def LF_other_relationship(c):
#     return (-1,1) if len(other.intersection(get_between_tokens(c))) > 0 else (0,0)


# import bz2

# # Function to remove special characters from text
# def strip_special(s):
#     return ''.join(c for c in s if ord(c) < 128)

# # Read in known spouse pairs and save as set of tuples
# with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'rb') as f:
#     known_spouses = set(
#         tuple(strip_special(x.decode('utf-8')).strip().split(',')) for x in f.readlines()
#     )
# # Last name pairs for known spouses
# last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
# def LF_distant_supervision(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

# def LF_distant_supervision_last_names(c):
#     p1, p2 = c.person1.get_span(), c.person2.get_span()
#     p1n, p2n = last_name(p1), last_name(p2)
#     return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,0)


# LFs = [
#     LF_distant_supervision, LF_distant_supervision_last_names, 
#     LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
#     LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
#     LF_family_left_window, LF_other_relationship
# ]

In [39]:
##### Continuous ################


import re
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)


spouses = {'spouse', 'wife', 'husband', 'ex-wife', 'ex-husband'}
family = {'father', 'mother', 'sister', 'brother', 'son', 'daughter',
              'grandfather', 'grandmother', 'uncle', 'aunt', 'cousin'}
family = family | {f + '-in-law' for f in family}
other = {'boyfriend', 'girlfriend' 'boss', 'employee', 'secretary', 'co-worker'}

# Helper function to get last name
def last_name(s):
    name_parts = s.split(' ')
    return name_parts[-1] if len(name_parts) > 1 else None    

def LF_husband_wife(c):
    global LF_Threshold
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for sw in spouses:
        sc=max(sc,get_similarity(word_vectors,sw))
    return (1,sc)

def LF_husband_wife_left_window(c):
    global LF_Threshold
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for sw in spouses:
        sc_1=max(sc_1,get_similarity(word_vectors,sw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for sw in spouses:
        sc_2=max(sc_2,get_similarity(word_vectors,sw))
    return(1,max(sc_1,sc_2))
    
def LF_same_last_name(c):
    p1_last_name = last_name(c.person1.get_span())
    p2_last_name = last_name(c.person2.get_span())
    if p1_last_name and p2_last_name and p1_last_name == p2_last_name:
        if c.person1.get_span() != c.person2.get_span():
            return (1,1)
    return (0,0)

def LF_no_spouse_in_sentence(c):
    return (-1,0.75) if np.random.rand() < 0.75 and len(spouses.intersection(c.get_parent().words)) == 0 else (0,0)

def LF_and_married(c):
    global LF_Threshold
    word_vectors = get_word_vectors(preprocess(get_right_tokens(c)))
    sc = get_similarity(word_vectors,'married')
    
    if 'and' in get_between_tokens(c):
        return (1,sc)
    else:
        return (0,0)

def LF_familial_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for fw in family:
        sc=max(sc,get_similarity(word_vectors,fw))
        
    return (-1,sc) 

def LF_family_left_window(c):
    sc_1 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[0])))
    for fw in family:
        sc_1=max(sc_1,get_similarity(word_vectors,fw))
        
    sc_2 = 0
    word_vectors = get_word_vectors(preprocess(get_left_tokens(c[1])))
    for fw in family:
        sc_2=max(sc_2,get_similarity(word_vectors,fw))
        
    return (-1,max(sc_1,sc_2))

def LF_other_relationship(c):
    sc = 0
    word_vectors = get_word_vectors(preprocess(get_between_tokens(c)))
    for ow in other:
        sc=max(sc,get_similarity(word_vectors,ow))
        
    return (-1,sc) 

# def LF_other_relationship_left_window(c):
#     sc = 0
#     word_vectors = get_word_vectors(preprocess(get_left_tokens(c)))
#     for ow in other:
#         sc=max(sc,get_similarity(word_vectors,ow))
#     return (-1,sc) 

import bz2

# Function to remove special characters from text
def strip_special(s):
    s = s.decode("utf-8") 
    return ''.join(c for c in s if (ord(c) < 128))


# Read in known spouse pairs and save as set of tuples
with bz2.BZ2File('data/spouses_dbpedia.csv.bz2', 'r') as f:
    known_spouses = set(
        tuple(strip_special(x).strip().split(',')) for x in f.readlines()
    )
# Last name pairs for known spouses
last_names = set([(last_name(x), last_name(y)) for x, y in known_spouses if last_name(x) and last_name(y)])
    
def LF_distant_supervision(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    return (1,1) if (p1, p2) in known_spouses or (p2, p1) in known_spouses else (0,0)

def LF_distant_supervision_last_names(c):
    p1, p2 = c.person1.get_span(), c.person2.get_span()
    p1n, p2n = last_name(p1), last_name(p2)
    return (1,1) if (p1 != p2) and ((p1n, p2n) in last_names or (p2n, p1n) in last_names) else (0,1)

import numpy as np

# def LF_Three_Lists_Left_Window(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife_left_window(c)
#     c2,s2 = LF_family_left_window(c)
#     c3,s3 = LF_other_relationship_left_window(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'Left:',s1,s2,s3,prob_sc
    
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1

# def LF_Three_Lists_Between_Words(c):
#     global softmax_Threshold
#     c1,s1 = LF_husband_wife(c)
#     c2,s2 = LF_familial_relationship(c)
#     c3,s3 = LF_other_relationship(c)
#     sc = np.array([s1,s2,s3])
#     c = [c1,c2,c3]
#     sharp_param = 1.5
    
#     prob_sc = np.exp(sc * sharp_param - np.max(sc))
#     prob_sc = prob_sc / np.sum(prob_sc)
#     #print 'BW:',s1,s2,s3,prob_sc
#     if s1==s2 or s3==s1:
#         return (0,0)
#     return c[np.argmax(prob_sc)],1
    
LFs = [
    LF_distant_supervision, LF_distant_supervision_last_names, 
    LF_husband_wife, LF_husband_wife_left_window, LF_same_last_name,
    LF_no_spouse_in_sentence, LF_and_married, LF_familial_relationship, 
    LF_family_left_window, LF_other_relationship
]

In [40]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [41]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

# dev_L_S = get_L_S_Tensor(dev_cands)

# np.save("dev_L_S_smooth",np.array(dev_L_S))

# train_L_S = get_L_S_Tensor(train_cands)
# np.save("train_L_S_smooth",np.array(train_L_S))


test_L_S = get_L_S_Tensor(test_cands)
np.save("test_L_S_smooth",np.array(test_L_S))



print("--- %s seconds ---" % (time.time() - start_time))



started at: 12-6-2018, 16:40:41
500data points labelled in 0.6098895152409871 mins
1000data points labelled in 1.2040858785311381 mins
1500data points labelled in 1.808166233698527 mins
2000data points labelled in 2.4930141886075337 mins
2500data points labelled in 3.091664159297943 mins
--- 203.35997462272644 seconds ---


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
    print("acc test",accuracy_score(gold_labels_dev,pl))
#     print("dev",precision_recall_fscore_support(gold_labels_dev,pl,average='binary'))
    print("test",precision_recall_fscore_support(gold_labels_test,pl,average='binary'))
#     print(confusion_matrix(gold_labels_dev,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
#     return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [44]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]
import numpy as np
dev_L_S = np.load("dev_L_S_smooth.npy")
test_L_S = np.load("test_L_S_smooth.npy")
train_L_S = np.load("train_L_S_smooth.npy")

# dev_L_S = np.load("dev_L_S_discrete.npy")
# train_L_S = np.load("train_L_S_discrete.npy")

print(test_L_S.shape,dev_L_S.shape,train_L_S.shape)

(2702, 2, 10) (2814, 2, 10) (22276, 2, 10)


In [45]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32
seed = 12

In [161]:
LF_l = [
    1,1,1,1,1,-1,1,-1,-1,-1
]

def merge(a,b):
    c = []
    for i in range(len(a)):
        ci = []
        ci_l = a[i,0,:].tolist()+b[i,0,:].tolist()
        ci_s = a[i,1,:].tolist()+b[i,1,:].tolist()
        ci.append(ci_l)
        ci.append(ci_s)
        c.append(ci)
    return c
import numpy as np
dev_L_S_s = np.load("dev_L_S_smooth.npy")
train_L_S_s = np.load("train_L_S_smooth.npy")

dev_L_S_d = np.load("dev_L_S_discrete.npy")
train_L_S_d = np.load("train_L_S_discrete.npy")

dev_L_S = np.array(merge(dev_L_S_d,dev_L_S_s))
train_L_S = np.array(merge(train_L_S_d,train_L_S_s))

LF_l = LF_l + LF_l
print(len(LF_l))


print(dev_L_S.shape,train_L_S.shape)

20
(2814, 2, 20) (22276, 2, 20)


In [163]:
NoOfLFs= len(LF_l)
NoOfClasses = 2
print(len(LF_l))

20


In [46]:
## normalized model with smooth LFs + penalties

class model:
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    
    def __init__(self,NoOfLFs,pcl=np.array([-1,1],dtype=np.float64),norm=True,smooth=True,penalty=0,p3k=3,alp=0.99):
        self.NoOfLFs = NoOfLFs
        self.norm = norm
        self.smooth = smooth
        self.p3k = p3k
        self.alp = alp
        self.pcl = pcl
        self.penalty = penalty
        self.seed = 12
        
    def build_graph(self):
        tf.set_random_seed(self.seed)
        
    
    
    BATCH_SIZE = 1
    tf.reset_default_graph()
    

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))

     
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=af,\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                                initializer=th,\
                        dtype=tf.float64)

#         print("thetas",thetas)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
        print(s)
        print("l",l)
#         print(s.graph)
        
        s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
        print("s_",s_)

       
        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            print("out",out)
            return out

        if(smooth):
            pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
        else:
            pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

        print("pout",pout)    

        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                           name="t_pout")
    
        print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        print("t",t)
        
        def ints(y):
            ky = iskequalsy(k,y)
            print("ky",ky)
            out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
            print("intsy",out1)
            return out1
                

        if(smooth):
            #smooth normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                           pcl,name="zy")
        else:
            #discrete normalizer
            zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                           pcl,name="zy")

    
# 
#         zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
#                        np.array(NoOfClasses,dtype=np.float64))
        
        print("zy",zy)
        logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
        
        print("logz",logz)
        tf.summary.scalar('logz', logz)
        lsp = tf.reduce_logsumexp(t_pout,axis=0)
        print("lsp",lsp)
        tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -tf.minimum( tf.reduce_min(thetas),0.0)
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +tf.reduce_sum(tf.log(1+tf.exp(-thetas-pk)))
        else:
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            
        print("loss",loss)
        tf.summary.scalar('un-normloss', loss)
#         tf.summary.histogram('thetas', t)
#         tf.summary.histogram('alphas', alphas)
#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

        print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(lr).minimize(loss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        summary_merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter('./summary/train',
                                      tf.get_default_graph())
        test_writer = tf.summary.FileWriter('./summary/test')

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        
        # ep : epochs
        # 
        def train(lr,ep,th,af,pcl=np.array([-1,1],dtype=np.float64),alp=0.99):
            
            with tf.Session() as sess:
                sess.run(init_g)
                sess.run(init_l)

                # Initialize an iterator over the training dataset.
                for en in range(ep):
                    sess.run(train_init_op)
                    tl = 0
                    try:
                        it = 0
                        while True:
                            sm,_,ls,t = sess.run([summary_merged,train_step,loss,thetas])
    #                         print(t)
    #                         print(tl)
                            train_writer.add_summary(sm, it)
    #                         if(ls<1e-5):
    #                             break
                            tl = tl + ls
                            it = it + 1

                    except tf.errors.OutOfRangeError:
                        pass
                    print(en,"loss",tl)

                    sess.run(dev_init_op)
                    sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
                    test_writer.add_summary(sm, en)
                    print(a)
                    print(t)
                    unique, counts = np.unique(pl, return_counts=True)
                    print(dict(zip(unique, counts)))
                    print("acc",accuracy_score(true_labels,pl))
                    print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))
                    print()

                # Initialize an iterator over the validation dataset.
                sess.run(dev_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                print(a)
                print(t)

                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))

                print("acc",accuracy_score(true_labels,pl))

                predictAndPrint(pl)
                print(precision_recall_fscore_support(np.array(true_labels),np.array(pl),average="binary"))

    #             cf = confusion_matrix(true_labels,pl)
    #             print(cf)
    return pl

SyntaxError: 'return' outside function (<ipython-input-46-d34e68709997>, line 226)

In [169]:
for i in np.linspace(0,1,11):
    print("alpha-mean",i)
    R=train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                                af = tf.truncated_normal_initializer(i,0.001,seed),
                              pcl=np.array([-1,1],dtype=np.float64),norm=True,smooth=True,penalty=0)

alpha-mean 0.0
Tensor("unstack:1", shape=(?, 20), dtype=float64)
l Tensor("unstack:0", shape=(?, 20), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 20), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 20), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(20,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(20,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(20,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(20,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
loss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 316296.1427546758
[ 0.00217267 -0.00087164  0.00096919  0.00090097  0.00204377 -0.00045195
  0.0026911

In [141]:
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.7,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),smooth=True,penalty=0,alp=0.8)

Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss 154419.85892351693
[0.70199849 0.69863948 0.70064995 0.70050634 0.70158368 0.69974811
 0.7024189  0.69989863 0.70116149 0.70031883]
[[1.11684702 0.81388424 1.00909

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [160]:
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.7,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),norm=False,\
                          smooth=True,penalty=2,alp=0.8)

Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
s_ Tensor("Maximum:0", shape=(?, 10), dtype=float64)
pout Tensor("pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("t_pout/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t Tensor("Squeeze:0", shape=(10,), dtype=float64)
out Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
ky Tensor("zy/while/Select:0", shape=(10,), dtype=float64)
intsy Tensor("zy/while/add:0", shape=(10,), dtype=float64)
zy Tensor("zy/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("logz:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(?, 1), dtype=float64)
unnormlized loss
Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
0 loss -15939.431711918833
[0.70114658 0.6980671  0.7005352  0.70036628 0.70097048 0.69970941
 0.70172842 0.6998

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [162]:
train(0.001/len(train_L_S),5,th = tf.truncated_normal_initializer(1,0.1,seed),\
                            af = tf.truncated_normal_initializer(0.7,0.001,seed),
                          pcl=np.array([-1,1],dtype=np.float64),norm=False,\
                          smooth=True,penalty=0,alp=2)

Tensor("unstack:1", shape=(?, 20), dtype=float64)
l Tensor("unstack:0", shape=(?, 20), dtype=float64)


ValueError: Dimensions must be equal, but are 20 and 10 for 'Sub' (op: 'Sub') with input shapes: [?,20], [10].